# I. Install neccessary library

In [1]:
pip install -qqq pandas numpy matplotlib plotly

Note: you may need to restart the kernel to use updated packages.


# II. Cleaning Data

## 1. Unzip files

In [2]:
import zipfile
import os
import shutil
from pathlib import Path
import os
import glob

In [3]:
titles = [
    "MedIT-RC CSQ Monthly",
    "MedIT-RC Agent State Detail Monthly",
    "MedIT-RC Agent Call Summary Monthly",
    "MedIT-RC Agent Monthly Report",
    "MedIT-RC Agent State Summary by Interval Monthly"
]

In [4]:
def runzip_and_group(sourcePath, destPath):
    source = Path(sourcePath)
    dest = Path(destPath)

    # Create flat folders for each title once
    title_folders = {title: (dest / title) for title in titles}
    for folder in title_folders.values():
        folder.mkdir(parents=True, exist_ok=True)

    for entry in source.rglob("*.zip"):
        zip_name = entry.stem

        for title in titles:
            if zip_name.startswith(title):
                target_folder = title_folders[title]

                with zipfile.ZipFile(entry, 'r') as zip_ref:
                    for file_info in zip_ref.infolist():
                        if not file_info.is_dir() and file_info.filename.lower().endswith(('.xls', '.xlsx')):
                            original_name = Path(file_info.filename).name
                            safe_name = f"{zip_name} - {original_name}"
                            dest_path = target_folder / safe_name

                            with zip_ref.open(file_info) as src_file, open(dest_path, 'wb') as out_file:
                                out_file.write(src_file.read())

                break

In [5]:
runzip_and_group("../data", "../data/unzip-files")

## 2. Merge files into Dataframe

In [6]:
source = Path("../data")
dest = Path("../data/unzip-files")

In [7]:
import pandas as pd

path_list = [(dest / title) for title in titles]
df_list = {}

for path in path_list:
    new_df = pd.DataFrame()
    for file in path.glob("*.xls"): 
        try:
            sub_df = pd.read_html(file)[0]
            new_df = pd.concat([new_df, sub_df], ignore_index=True)
        except Exception as e:
            print(f"Skipped {file.name} due to error: {e}")

    df_list[path.name] = new_df

print(f'There are {len(df_list)} dataframes. There key names are: {df_list.keys()}')

There are 5 dataframes. There key names are: dict_keys(['MedIT-RC CSQ Monthly', 'MedIT-RC Agent State Detail Monthly', 'MedIT-RC Agent Call Summary Monthly', 'MedIT-RC Agent Monthly Report', 'MedIT-RC Agent State Summary by Interval Monthly'])


## 3. Dataframe's dimension

In [8]:
for title, dataframe in df_list.items():
    print(f"This {title} dataframe has {dataframe.shape[0]} rows and {dataframe.shape[1]} columns.")

This MedIT-RC CSQ Monthly dataframe has 2141 rows and 18 columns.
This MedIT-RC Agent State Detail Monthly dataframe has 4222 rows and 15 columns.
This MedIT-RC Agent Call Summary Monthly dataframe has 209 rows and 24 columns.
This MedIT-RC Agent Monthly Report dataframe has 4216 rows and 15 columns.
This MedIT-RC Agent State Summary by Interval Monthly dataframe has 200 rows and 15 columns.


## 5. POXIS time conversion and extracting time information

In [9]:
df_list["MedIT-RC Agent Monthly Report"].head()

,Agent Name,Agent ID,Extension,Call Start Time,Call End Time,Duration,Called Number,Call ANI,Call Routed CSQ,Other CSQs,Call Skills,Talk Time,Hold Time,Work Time,Call Type
0,Akshay Kumar,29124.0,NaN,10/12/23 9:39:39 AM,10/12/23 9:42:00 AM,00:02:21,9.236890e+10,29124,NaN,NaN,NaN,00:02:21,NaN,NaN,Outbound on Non-IPCC
1,Akshay Kumar,29124.0,83703.0,10/12/23 10:54:54 AM,10/12/23 10:54:59 AM,00:00:05,8.703400e+04,6048228991,MedIT - RC,NaN,sk_MedIT_RC,00:00:00,00:00:00,00:00:00,Inbound ACD
2,Akshay Kumar,29124.0,83703.0,10/12/23 11:00:04 AM,10/12/23 11:00:23 AM,00:00:19,8.703400e+04,6048228991,MedIT - RC,NaN,sk_MedIT_RC,00:00:00,00:00:00,00:00:00,Inbound ACD
3,Akshay Kumar,29124.0,83703.0,10/12/23 4:20:31 PM,10/12/23 4:23:56 PM,00:03:25,8.703400e+04,7785499997,MedIT - RC,NaN,sk_MedIT_RC,00:03:14,00:00:00,00:00:45,Inbound ACD
4,Akshay Kumar,29124.0,83703.0,10/17/23 12:42:47 PM,10/17/23 12:44:57 PM,00:02:10,8.703400e+04,6047855563,MedIT - RC,NaN,sk_MedIT_RC,00:01:58,00:00:00,00:00:45,Inbound ACD


In [10]:
df_list["MedIT-RC Agent State Detail Monthly"].head()

,Agent Name,Agent ID,Extension,Call Start Time,Call End Time,Duration,Called Number,Call ANI,Call Routed CSQ,Other CSQs,Call Skills,Talk Time,Hold Time,Work Time,Call Type
0,Akshay Kumar,29124.0,83703.0,10/28/24 3:21:08 PM,10/28/24 3:21:26 PM,00:00:18,87034.0,2509974626,MedIT - RC,NaN,sk_MedIT_RC,00:00:00,00:00:00,00:00:00,Inbound ACD
1,Akshay Kumar,29124.0,83703.0,10/29/24 9:33:15 AM,10/29/24 9:35:10 AM,00:01:55,87034.0,4039512209,MedIT - RC,NaN,sk_MedIT_RC,00:01:44,00:00:00,00:00:45,Inbound ACD
2,Akshay Kumar,29124.0,83703.0,10/29/24 1:26:37 PM,10/29/24 1:28:54 PM,00:02:17,87034.0,4039512209,MedIT - RC,NaN,sk_MedIT_RC,00:02:07,00:00:00,00:00:45,Inbound ACD
3,Akshay Kumar,29124.0,83703.0,10/30/24 11:10:08 AM,10/30/24 11:10:26 AM,00:00:18,87034.0,6048754411,MedIT - RC,NaN,sk_MedIT_RC,00:00:00,00:00:00,00:00:00,Inbound ACD
4,Akshay Kumar,29124.0,83703.0,10/30/24 12:19:44 PM,10/30/24 12:21:47 PM,00:02:03,87034.0,6048068397,MedIT - RC,NaN,sk_MedIT_RC,00:01:52,00:00:00,00:00:45,Inbound ACD


In [11]:
df_agent_monthly_report = df_list["MedIT-RC Agent Monthly Report"]
df_agent_state_detail_monthly = df_list["MedIT-RC Agent State Detail Monthly"]

In [12]:
for dataframe in [df_agent_monthly_report, df_agent_state_detail_monthly]:
    for col in [3, 4]:
        col_name = dataframe.columns[col]
        dataframe[col_name] = pd.to_datetime(
            dataframe[col_name],
            format="%m/%d/%y %I:%M:%S %p",
            errors="coerce"
        )
    dataframe["Year"] = dataframe.iloc[:, 3].dt.year.fillna(0).astype(int)
    dataframe["Month"] = dataframe.iloc[:, 3].dt.month.fillna(0).astype(int)
    dataframe["Day"] = dataframe.iloc[:, 3].dt.day.fillna(0).astype(int)
    dataframe["Day-Name"] = dataframe.iloc[:, 3].dt.day_name().fillna("Unknown")

In [13]:
df_agent_monthly_report.head()

,Agent Name,Agent ID,Extension,Call Start Time,Call End Time,Duration,Called Number,Call ANI,Call Routed CSQ,Other CSQs,Call Skills,Talk Time,Hold Time,Work Time,Call Type,Year,Month,Day,Day-Name
0,Akshay Kumar,29124.0,NaN,2023-10-12 09:39:39,2023-10-12 09:42:00,00:02:21,9.236890e+10,29124,NaN,NaN,NaN,00:02:21,NaN,NaN,Outbound on Non-IPCC,2023,10,12,Thursday
1,Akshay Kumar,29124.0,83703.0,2023-10-12 10:54:54,2023-10-12 10:54:59,00:00:05,8.703400e+04,6048228991,MedIT - RC,NaN,sk_MedIT_RC,00:00:00,00:00:00,00:00:00,Inbound ACD,2023,10,12,Thursday
2,Akshay Kumar,29124.0,83703.0,2023-10-12 11:00:04,2023-10-12 11:00:23,00:00:19,8.703400e+04,6048228991,MedIT - RC,NaN,sk_MedIT_RC,00:00:00,00:00:00,00:00:00,Inbound ACD,2023,10,12,Thursday
3,Akshay Kumar,29124.0,83703.0,2023-10-12 16:20:31,2023-10-12 16:23:56,00:03:25,8.703400e+04,7785499997,MedIT - RC,NaN,sk_MedIT_RC,00:03:14,00:00:00,00:00:45,Inbound ACD,2023,10,12,Thursday
4,Akshay Kumar,29124.0,83703.0,2023-10-17 12:42:47,2023-10-17 12:44:57,00:02:10,8.703400e+04,6047855563,MedIT - RC,NaN,sk_MedIT_RC,00:01:58,00:00:00,00:00:45,Inbound ACD,2023,10,17,Tuesday


In [14]:
df_agent_monthly_report.head()

,Agent Name,Agent ID,Extension,Call Start Time,Call End Time,Duration,Called Number,Call ANI,Call Routed CSQ,Other CSQs,Call Skills,Talk Time,Hold Time,Work Time,Call Type,Year,Month,Day,Day-Name
0,Akshay Kumar,29124.0,NaN,2023-10-12 09:39:39,2023-10-12 09:42:00,00:02:21,9.236890e+10,29124,NaN,NaN,NaN,00:02:21,NaN,NaN,Outbound on Non-IPCC,2023,10,12,Thursday
1,Akshay Kumar,29124.0,83703.0,2023-10-12 10:54:54,2023-10-12 10:54:59,00:00:05,8.703400e+04,6048228991,MedIT - RC,NaN,sk_MedIT_RC,00:00:00,00:00:00,00:00:00,Inbound ACD,2023,10,12,Thursday
2,Akshay Kumar,29124.0,83703.0,2023-10-12 11:00:04,2023-10-12 11:00:23,00:00:19,8.703400e+04,6048228991,MedIT - RC,NaN,sk_MedIT_RC,00:00:00,00:00:00,00:00:00,Inbound ACD,2023,10,12,Thursday
3,Akshay Kumar,29124.0,83703.0,2023-10-12 16:20:31,2023-10-12 16:23:56,00:03:25,8.703400e+04,7785499997,MedIT - RC,NaN,sk_MedIT_RC,00:03:14,00:00:00,00:00:45,Inbound ACD,2023,10,12,Thursday
4,Akshay Kumar,29124.0,83703.0,2023-10-17 12:42:47,2023-10-17 12:44:57,00:02:10,8.703400e+04,6047855563,MedIT - RC,NaN,sk_MedIT_RC,00:01:58,00:00:00,00:00:45,Inbound ACD,2023,10,17,Tuesday


# III. Saving all the files

In [15]:
output_folder = "../data/semi-cleaned-data"

for key, dataframe in df_list.items():
    file_path = os.path.join(output_folder, f"{key}.csv")
    if not os.path.exists(file_path):
        dataframe.to_csv(file_path, index=False)

## IV. Cleaning up and delete files:

In [16]:
folder_path = "../data/unzip-files"

# Delete all files in the folder
for file_path in glob.glob(os.path.join(folder_path, "*")):
    if os.path.isfile(file_path):
        os.remove(file_path)